In [1]:
import twint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
pd.options.display.max_columns = 40


In [3]:
import datetime as dt
import time
import random

def arrange_dates(start_date, end_date):
    """Used in the get_election_tweets func. """
    
    dates = []
    delta = end_date - start_date
    
    for i in range(delta.days + 1):
        day = start_date + dt.timedelta(days=i)
        dates.append(day)
    
    return dates

def get_election_tweets(tweets_per_day=5000, start_date="2020-10-01", end_date="2020-11-03"):
    
    # get a list of dates
    f = "%Y-%m-%d"
    sd = dt.datetime.strptime(start_date, f)
    ed = dt.datetime.strptime(end_date, f)
    dates = arrange_dates(sd, ed)
    candidates = ['Biden', 'Trump']
    
    # fetch tweets & save CSVs (each CSV contains one day/candidate)
    
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 60 # seconds; will use for sleep timer to avoid 'client payload' error
    
    for date in dates:
        for candidate in candidates:
            
            d = date.strftime("%Y-%m-%d")            
            print(f"""\n Begin scraping {candidate}'s tweets on {d}\n \n""")
            outfile = f"./pickle/tweets/{candidate}_{d}.csv"
            
            # twint search query details
            c = twint.Config()
            c.Search = candidate
            c.Lang = 'en'
            c.Since = d
            c.Until = lam_plus_1(date)
            c.Limit = tweets_per_day
            c.Store_csv = True
            c.Output = outfile
            
            # run the search
            twint.run.Search(c)
            print(f"Scraping complete. File: {outfile} created.")
            
            # wait some time so script does not receive 'client payload' error
            print(f"Waiting 60-100 seconds...")
            time.sleep(wait_time + random.choice([5, 10, 15, 20, 25, 30]) + (20 * random.uniform(0, 1)))
                       

In [ ]:
get_election_tweets(start_date="2020-10-26", end_date="2020-11-03")


 Begin scraping Biden's tweets on 2020-10-26
 

1320877870554763264 2020-10-26 23:59:59 +0000 <indiegop> @RichardGrenell @maryannemarsh Please.  A 100 million was spent on voter contact and an enormous number of contacts were made.  Biden didn’t start this until a month ago.
1320877870026248192 2020-10-26 23:59:59 +0000 <Swampdo01077548> Thank you @SenSchumer @JoeBiden and Harry Reid!  Without your new rules on SCOTUS nominations in an election year we wouldn't have done it!  #AmyConeyBarrett @senatemajldr @GOP @FDRLST #WinningWinningWinning  https://t.co/ovydFD9CXA
1320877869988536320 2020-10-26 23:59:59 +0000 <lisajew62> @realDonaldTrump LIES. The only problems we are having is due to your interference. Its not Biden who can “only win if he cheats.” It’s you. We are aware of your little projection problem. The only people who believe you are sycophants and your supporters. And we all know why. #Racists
1320877869220962304 2020-10-26 23:59:59 +0000 <CethEslick> @JerryDunleavy Remembe

1320877810102243328 2020-10-26 23:59:45 +0000 <PChernick> @ValerieJarrett AND vote for @JoeBiden.  That’s patriotism too!!
1320877809678635009 2020-10-26 23:59:45 +0000 <SergeantPinback> Biden's Moment  https://t.co/84PDYC3l7f via @YouTube
1320877808864894977 2020-10-26 23:59:45 +0000 <Kasia24132094> Trump wavers between ‘insulted and happy’ after Biden confuses him with ...  https://t.co/LTpIN25mZ0 via @YouTube
1320877808457928706 2020-10-26 23:59:44 +0000 <IamBradMinoski> @jkempcpa No. If Biden wants any chance of having his legislation passed and preserve our rights and democracy then he has no choice but to expand the courts. You can’t keep taking the high road when the other side gleefully kicks dirt on your face at every opportunity.
1320877808084766723 2020-10-26 23:59:44 +0000 <iAmTwicky> @TerreekMaBra I ain’t protecting no one I ain’t even voting I just asking. Tbh if I vote I would be voting for my union and my union supports Biden.
1320877807464042504 2020-10-26 23:59:44 +00

1320877732436283393 2020-10-26 23:59:26 +0000 <auntmimihere> Biden said, “it was a mistake and xenophobic” when DJT halted flights from China. After all, he had used his VP position as form of currency &amp; certainly didn’t want to offend the Chinese.
1320877731073110016 2020-10-26 23:59:26 +0000 <RickSmithShow> Trump said Biden is the worst candidate ever.  "I don't think he's going to make it."  He's terrible. He's this.  He's that.  HE IS KICKING YOUR ASS
1320877730486013952 2020-10-26 23:59:26 +0000 <ManDeer666> What is Joe Biden’s plan for America? Just wondering.
1320877730179809280 2020-10-26 23:59:26 +0000 <MissAnn88242695> @JoeBiden @MichelleObama Like anyone listens to her, outside the sheeple slave mentality followers of hers. #BLEXIT
1320877729458212865 2020-10-26 23:59:26 +0000 <StefanK1117> @TeamTrump @realDonaldTrump If Joe Biden is elected he will deliver a safe vaccine, deliver therapies, stop ignoring the pandemic, open your schools safely, and safe your country from

1320877659921014797 2020-10-26 23:59:09 +0000 <hrprogressive> Hey @JoeBiden and Every Fucking Democrat.  When Biden wins resoundingly.  Mitch Mcconnell is literally saying elections have consequences.  Therefore,  #ExpandTheCourt is legitimate.   It is the only route to a legitimate #SCOTUS
1320877659912499200 2020-10-26 23:59:09 +0000 <HeleneShaw17> CHINA.   They are blackmailing Joe Biden... as if they're not doing it to Justin the perv.  Suspects in alleged Markham illegal casino mansion linked to B.C. casino suspects |  https://t.co/SM4d7CgpxR  https://t.co/BFFTDGvjvO
1320877659853901825 2020-10-26 23:59:09 +0000 <Jackrfly1> @JoeBiden must watch tv tonight at 8. See the truth!
1320877659509936128 2020-10-26 23:59:09 +0000 <MarkBiggsSr> @realDonaldTrump Cheat on your wives, cheat on your taxes, why not try and cheat America out of their votes!   That's what a cheater would do! Thanks again for reminding me to vote for Joe Biden!  #BidenHarris2020  #BidenHarrisLandslide2020
132087765

1320877598570876929 2020-10-26 23:58:54 +0000 <CherylsellsKaty> @realDonaldTrump you failed the American people! #Vote BIDEN HARRIS-dump trump #TrumpVirusDeathToll220K  #TrumpHatesOurTroops #TrumpDestroysdemocracy #TrumpLiedPeopleDied #VoteForDemocracy  #TrumpLiedPeopleDied #TrumpProjection #Letswinthis #GOPSuperSpreaders #TooBigToRig  https://t.co/JpVQwThytt
1320877598260514816 2020-10-26 23:58:54 +0000 <SleepyCongress1> battleground states w 8 days out  CO Biden +13.7 FL Biden +2.3 IA Biden +1.2 MI Biden +8.3 MN Biden +8.1 NH Biden +11.2 NC Biden +2.9 OH Trump +1.6 PA Biden +5.1  TX Trump +1.2 VA Biden +11.3  WI Biden +7.1   https://t.co/WIGyBoSIkE
1320877598000484352 2020-10-26 23:58:54 +0000 <HisDudeness72> @GrotesqueLogic @bobpi31415 @JoeBiden And what are you hanging this certainty on?  Your gut?  I hate to break it to you, but Texas has been trending Democrat for several years now.  It's why O'Rourke came close to unseating Cruz.  Which is why I think there is definitely a chanc

...

1320874559495675905 2020-10-26 23:46:50 +0000 <BrEldarion> @GabeHoff @realryanhartwig Biden pledges to end all the USA main values
1320874558338011136 2020-10-26 23:46:50 +0000 <serendipity191> Stop the super spreader in chief #voteblue  vote @JoeBiden @KamalaHarris
1320874558128312320 2020-10-26 23:46:49 +0000 <ThisIsEricsTrip> @Zach21722958 @gopcausecancer @AlbertLevy0105 @JoeBiden Keep that same energy next time Dems control Congress, Zachary. Everything you've said will work splendidly as justification for an expansion of the courts.
1320874557515943937 2020-10-26 23:46:49 +0000 <Robb_Wexler> @TheHRH He's talking to GEORGE Lopez. I have now seen that done on "Zoom" calls hundreds of times..i.e..where you want to mention a name and all you see is the person you are speaking with and use their name. Biden here mentions Trump a second afterwards. Again, WTH is wrong with you?
1320874557117501442 2020-10-26 23:46:49 +0000 <abracadabra150> @ComfortablySmug Mitch should retire when Biden

1320874502419603459 2020-10-26 23:46:36 +0000 <silkscreener> @JohnKaminar @Memers08 @JoeBiden @KamalaHarris @xjelliott Thank you sir for your service to our country and for trying to save our country once again
1320874502197305344 2020-10-26 23:46:36 +0000 <DArcouette> @realDonaldTrump Absolutely! That's why people need to vote Biden
1320874501878370305 2020-10-26 23:46:36 +0000 <geodesic29> @DailyCaller Bye Bye Bye Bye Biden Biden Good Bye.
1320874501832429570 2020-10-26 23:46:36 +0000 <markwheeler42> @KamalaHarris @CheriJacobus I voted Biden Harris
1320874500880216064 2020-10-26 23:46:36 +0000 <RobertFouquet> @seanmdav The left  on whistleblower’s with no evidence against Trump:  “we must believe whistleblowers no matter what! They must be protected, respected, and believed”  The left  on whistleblower’s WITH evidence against Biden: “LIAR! There’s nothing to see here. Russian propaganda!”
1320874500737634304 2020-10-26 23:46:36 +0000 <shawnbennett223> @RealJamesWoods I really hope Bi

1320874435918876672 2020-10-26 23:46:20 +0000 <ChipmunkWhispe1> @a_mar_hum @Julio_1thousand @TaceyJean18 @felon_fred @JoeBiden These days, I try to find as many "Hahahahaha" moments that I can. I gave myself one yesterday making a stupid 45 second video about the extinction of today's republican party. 🤪
1320874435771940864 2020-10-26 23:46:20 +0000 <aheathcetinski> @glubold @daveweigel This was not one for Biden. Watch the clip. He was addressing the interviewer (named George.)
1320874435285536768 2020-10-26 23:46:20 +0000 <AdvocateErin> @grandmagrit @JoeBiden Yep. Last week. As did my husband and daughters boyfriend that just turned 18.
1320874435100971010 2020-10-26 23:46:20 +0000 <teapartytempest> Russia and China want Biden to be president becasue they can go back to running over the USA.
1320874434555715586 2020-10-26 23:46:20 +0000 <IrishPatriot19> @SimakowiczAlma @basford_h @EricTrump Remember this!😡 I certainly do!!! Biden is lying racist and thief of our tax dollars😡  https:/

1320874373515980801 2020-10-26 23:46:05 +0000 <Monieaba> Here I am on @HillaryClinton 's birthday thinking, there couldn't have possibly been a worse candidate than her. But then @JoeBiden stepped up and said hold my, you know, the thing
1320874372593295360 2020-10-26 23:46:05 +0000 <LetsBeFranc> @ShaneGoguen That's just how politician's role on the campaign trail. Look at Biden/Harris' policies. It's clear they would be better leaders for the US. Also, the likelihood is that Joe will step down in 2 years due to ill health and Harris will become the first woman president of America.
1320874371703918592 2020-10-26 23:46:05 +0000 <Jason> 538 &amp; Economist screaming @JoeBiden   ... what do you think?  https://t.co/xxukyu89tV
1320874370877804544 2020-10-26 23:46:05 +0000 <101_sensitive> @realDonaldTrump Remember my fellow Americans... A vote for @JoeBiden is a vote for CHINESE SUPREMACY!
1320874370244530180 2020-10-26 23:46:05 +0000 <cafenetamerica> @Lizbethjoy @Chicago1Ray @TheRISEofROD

1320874304532348929 2020-10-26 23:45:49 +0000 <eifers1> @jonathans_tobin @realDonaldTrump @JNS_org @JoeBiden Dems and media have been swiping at trump for four years. There’s isn’t a swipe back that could be unjustified.
1320874303445966853 2020-10-26 23:45:49 +0000 <52fairway> Dying dad, lifelong conservative, votes for Biden out of respect for his six daughters #SmartNews   https://t.co/8UzdL8RZVL
1320874303156449282 2020-10-26 23:45:49 +0000 <hobobaginsesq> @caitiedelaney Biden will say anything you want to hear. He's like a fish out of water flip flop flip flop flip flop
1320874301348827136 2020-10-26 23:45:48 +0000 <eliseb> Biden supporters: volunteers are urgently needed to help call NC voters whose absentee ballots have been rejected and help them fix it. No prior volunteer experience needed. 🙏🏻 #BidenHarris2020 #WomenForBiden   https://t.co/MPAVAfINs7
1320874300765851650 2020-10-26 23:45:48 +0000 <jet7111> @braves52 @CNN Nope they have no interest in this at all because it's te